In [43]:
%reload_ext autoreload
%autoreload 2
%config Completer.use_jedi=False

from data import *
from models import *
import sample_model
import pytorch_lightning as pl

In [44]:
dataset = 'levin'
model   = 'sample_model'
epochs  = 2
lr      = 0.001

datasets_dict = {'levin':'sample_levin_dataset',
                 'GOPRO': None,
                 'lai': None}

models_dict   = {'sample_model': SampleModel}

dataset_info = datasets_dict.get(dataset, dataset)
name = dataset
if type(dataset_info) == str:
    data_module = DeblurDataModule(dataset_info, name=name)
elif type(dataset_info) == list:
    data_module = DeblurDataModule(*dataset_info, name=name)
elif type(dataset_info) == type(lambda x: x):
    train_files, test_files = dataset_info()
    data_module = DeblurDataModule(train_files=train_files, test_files=test_files, name=name)
else:
    raise Exception(f"Could not load dataset with argument: {dataset}")

    
model = SampleModel(data_module)

logger = pl.loggers.TensorBoardLogger(save_dir='experiments', name=model.hparams['exp_name'], 
                                      log_graph=False)
logger.log_hyperparams(params=model.hparams,
                       metrics={"PSNR":0.0, "SSIM": 0.0, "val_loss":0.0,
                                "epoch_test_PSNR":0.0, "epoch_test_SSIM":0.0})
trainer = pl.Trainer(logger, max_epochs=5)

Calculating dataset stats with a 50% random subset of training data


GPU available: False, used: False
TPU available: False, using: 0 TPU cores


# Model Fitting

In [4]:
trainer.fit(model, datamodule=data_module)


  | Name      | Type    | Params
--------------------------------------
0 | loss_func | MSELoss | 0     


Saving latest checkpoint..


1

In [5]:
logger.finalize('success')

# Saving and loading stuff